## Load facebook_combined (all nodes with edges)


In [ ]:
pip install node2vec networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy==1.23.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 104.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
node2vec 0.5.0 requires numpy<2.0.0,>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1

In [ ]:
import pandas as pd  # Make sure to import pandas

# Read the path of the txt file
file_path = "/content/drive/Shareddrives/MLNS_Final_Facebook/data/facebook_combined.txt/facebook_combined.txt"

# Read data (assuming it's an edge list separated by spaces)
df = pd.read_csv(file_path, sep=" ", header=None, names=["source", "target"])

# Add an 'edges' column with value 1
df["edges"] = 1

# Display the first few rows of the data
print(df.head())  # Display the first 5 rows
print(df.info())  # Display dataset summary information

   source  target  edges
0       0       1      1
1       0       2      1
2       0       3      1
3       0       4      1
4       0       5      1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88234 entries, 0 to 88233
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   source  88234 non-null  int64
 1   target  88234 non-null  int64
 2   edges   88234 non-null  int64
dtypes: int64(3)
memory usage: 2.0 MB
None


In [ ]:
import pandas as pd
import numpy as np

# Get all unique nodes
nodes = set(df["source"]).union(set(df["target"]))

# Store the set of positive sample edges (undirected edges, ensure (a, b) and (b, a) are not duplicated)
existing_edges = set(tuple(sorted(edge)) for edge in zip(df["source"], df["target"]))

# Generate strict 1:1 negative samples
negative_samples = set()
while len(negative_samples) < len(df):
    u, v = np.random.choice(list(nodes), 2, replace=False)  # Randomly select two different nodes
    edge = tuple(sorted((u, v)))  # Ensure (u, v) and (v, u) are treated as the same edge
    if edge not in existing_edges and edge not in negative_samples:
        negative_samples.add(edge)

# Create negative sample DataFrame
neg_df = pd.DataFrame(list(negative_samples), columns=["source", "target"])
neg_df["edges"] = 0  # Negative sample edges are labeled as 0

# Combine positive and negative samples
df_final = pd.concat([df, neg_df], ignore_index=True)

# Display DataFrame information
print(df_final.info())

# Preview the first 10 rows
print(df_final.head(10))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176468 entries, 0 to 176467
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   source  176468 non-null  int64
 1   target  176468 non-null  int64
 2   edges   176468 non-null  int64
dtypes: int64(3)
memory usage: 4.0 MB
None
   source  target  edges
0       0       1      1
1       0       2      1
2       0       3      1
3       0       4      1
4       0       5      1
5       0       6      1
6       0       7      1
7       0       8      1
8       0       9      1
9       0      10      1


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Separate positive and negative samples
df_positive = df_final[df_final["edges"] == 1]  # Positive samples
df_negative = df_final[df_final["edges"] == 0]  # Negative samples

# Split dataset with a 9:1 ratio (maintain 1:1 negative sampling)
train_pos, test_pos = train_test_split(df_positive, test_size=0.1, random_state=42)
train_neg, test_neg = train_test_split(df_negative, test_size=0.1, random_state=42)

# Combine training and test sets
df_train = pd.concat([train_pos, train_neg], ignore_index=True)
df_test = pd.concat([test_pos, test_neg], ignore_index=True)

# Print dataset split information
print(f"Training set samples: {len(df_train)} (Positive samples: {len(train_pos)}, Negative samples: {len(train_neg)})")
print(f"Test set samples: {len(df_test)} (Positive samples: {len(test_pos)}, Negative samples: {len(test_neg)})")

# Preview data
print("\nFirst 10 rows of the training set:")
print(df_train.head(10))

print("\nFirst 10 rows of the test set:")
print(df_test.head(10))

Training set samples: 158820 (Positive samples: 79410, Negative samples: 79410)
Test set samples: 17648 (Positive samples: 8824, Negative samples: 8824)

First 10 rows of the training set:
   source  target  edges
0    1945    2071      1
1     224     322      1
2    1833    1902      1
3    3226    3390      1
4    2835    2956      1
5    1055    1585      1
6    1600    1621      1
7    2873    3022      1
8    2409    2552      1
9     128     199      1

First 10 rows of the test set:
   source  target  edges
0    1188    1366      1
1    1496    1630      1
2    1958    2645      1
3    3204    3280      1
4     946    1181      1
5    1975    2403      1
6    3713    3867      1
7    2796    3434      1
8    2332    2399      1
9    1284    1587      1


In [ ]:
# Check class distribution in the training set
train_edges_counts = df_train["edges"].value_counts()
print("\nTraining set class distribution:")
print(f"Positive samples (edges=1): {train_edges_counts.get(1, 0)}")
print(f"Negative samples (edges=0): {train_edges_counts.get(0, 0)}")

# Check class distribution in the test set
test_edges_counts = df_test["edges"].value_counts()
print("\nTest set class distribution:")
print(f"Positive samples (edges=1): {test_edges_counts.get(1, 0)}")
print(f"Negative samples (edges=0): {test_edges_counts.get(0, 0)}")



Training set class distribution:
Positive samples (edges=1): 79410
Negative samples (edges=0): 79410

Test set class distribution:
Positive samples (edges=1): 8824
Negative samples (edges=0): 8824


## Read all circles files


In [ ]:
import os

# Set the path to the `facebook` directory
folder_path = r"/content/drive/Shareddrives/MLNS_Final_Facebook/data/facebook/facebook"

# Store social circle information for all users
circle_dict = {}

# Iterate through all `*.circles` files in the `facebook` directory
for file in os.listdir(folder_path):
    if file.endswith(".circles"):  # Process only `.circles` files
        ego_id = file.split(".")[0]  # Get the Ego ID from the filename
        file_path = os.path.join(folder_path, file)

        # Read the content of the `circles` file
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split()
                circle_name = f"{ego_id}_{parts[0]}"  # **Ensure circle name is unique**
                members = set(map(int, parts[1:]))  # Remaining values are the members of the circle

                # Record circle information
                for member in members:
                    if member not in circle_dict:
                        circle_dict[member] = set()
                    circle_dict[member].add(circle_name)  # Store unique circle names

print(f"Successfully loaded {len(circle_dict)} users' circle information!")

Successfully loaded 2884 users' circle information!


In [ ]:
from pprint import pprint

pprint(circle_dict)

{0: {'107_circle3'},
 1: {'0_circle15'},
 2: {'0_circle10'},
 3: {'0_circle15'},
 5: {'0_circle16'},
 6: {'0_circle15'},
 7: {'0_circle15'},
 9: {'0_circle16', '0_circle15'},
 10: {'0_circle15'},
 12: {'0_circle21'},
 13: {'0_circle11'},
 14: {'0_circle10'},
 15: {'0_circle15'},
 16: {'0_circle15'},
 17: {'0_circle19', '0_circle6'},
 20: {'0_circle19', '0_circle6'},
 21: {'0_circle15'},
 22: {'0_circle15'},
 23: {'0_circle5', '0_circle15'},
 24: {'0_circle16'},
 25: {'0_circle15'},
 26: {'0_circle15'},
 28: {'0_circle23'},
 29: {'0_circle0'},
 31: {'0_circle15'},
 32: {'0_circle6'},
 33: {'0_circle19'},
 34: {'0_circle16', '348_circle4', '348_circle11'},
 35: {'0_circle6'},
 36: {'0_circle16', '0_circle15'},
 37: {'0_circle16'},
 38: {'0_circle15'},
 39: {'0_circle15'},
 40: {'0_circle15'},
 41: {'0_circle19', '0_circle6'},
 42: {'0_circle10'},
 44: {'0_circle19'},
 45: {'0_circle15'},
 46: {'0_circle7'},
 47: {'0_circle15'},
 48: {'0_circle16'},
 49: {'0_circle15'},
 50: {'0_circle15'

## Construct circles features


In [ ]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict

# Calculate the number of shared social circles between users u and v
def get_circle_overlap(u, v):
    """Calculate the number of shared social circles between user u and v"""
    if u in circle_dict and v in circle_dict:
        return len(circle_dict[u] & circle_dict[v])  # Count the number of common circles
    return 0

# Calculate the number of social circles each user belongs to
user_circle_count = {user: len(circles) for user, circles in circle_dict.items()}

# Calculate the size of each social circle
circle_size = defaultdict(int)
for user, circles in circle_dict.items():
    for circle in circles:
        circle_size[circle] += 1

# Calculate the Jaccard similarity between users u and v based on circles
def circle_jaccard_similarity(u, v):
    """Compute the Jaccard similarity between user u and v (based on social circles)"""
    if u in circle_dict and v in circle_dict:
        intersection = len(circle_dict[u] & circle_dict[v])
        union = len(circle_dict[u] | circle_dict[v])
        return intersection / union if union > 0 else 0
    return 0

# Determine whether two users share the same "core" circle
def has_shared_core_circle(u, v):
    """Check whether u and v are in the same core social circle (core: size > 10)"""
    if u in circle_dict and v in circle_dict:
        shared_circles = circle_dict[u] & circle_dict[v]
        return int(any(circle_size.get(circle, 0) > 10 for circle in shared_circles))  # Directly check circle size
    return 0

# Compute the Adamic-Adar Index between two users (based on social circles)
def circle_adamic_adar_index(u, v):
    """Calculate the Adamic-Adar Index between u and v"""
    if u in circle_dict and v in circle_dict:
        shared_circles = circle_dict[u] & circle_dict[v]
        return sum(1 / np.log(circle_size.get(circle, 2)) for circle in shared_circles if circle_size.get(circle, 2) > 1)
    return 0

In [ ]:
# Define a function to add features
def add_circle_features(df):
    df["circle_overlap"] = df.apply(lambda row: get_circle_overlap(row["source"], row["target"]), axis=1)
    df["circle_jaccard_similarity"] = df.apply(lambda row: circle_jaccard_similarity(row["source"], row["target"]), axis=1)
    df["has_shared_core_circle"] = df.apply(lambda row: has_shared_core_circle(row["source"], row["target"]), axis=1)
    df["circle_adamic_adar_index"] = df.apply(lambda row: circle_adamic_adar_index(row["source"], row["target"]), axis=1)
    df["source_circle_count"] = df["source"].map(user_circle_count).fillna(0)
    df["target_circle_count"] = df["target"].map(user_circle_count).fillna(0)
    return df

# Apply feature engineering to the training and test sets
df_train = add_circle_features(df_train)
df_test = add_circle_features(df_test)

# Print the enhanced training and test set information
print("\nUpdated training set:")
print(df_train.head())

print("\nUpdated test set:")
print(df_test.head())


Updated training set:
   source  target  edges  circle_overlap  circle_jaccard_similarity  \
0    1945    2071      1               1                        0.5   
1     224     322      1               1                        1.0   
2    1833    1902      1               1                        1.0   
3    3226    3390      1               1                        1.0   
4    2835    2956      1               1                        1.0   

   has_shared_core_circle  circle_adamic_adar_index  source_circle_count  \
0                       1                  0.204484                  2.0   
1                       1                  0.204484                  1.0   
2                       1                  0.174517                  1.0   
3                       1                  0.184635                  1.0   
4                       1                  0.184635                  1.0   

   target_circle_count  
0                  1.0  
1                  1.0  
2                 

In [ ]:
# Check for missing values in df_train
print("\nMissing values in df_train:")
print(df_train.isnull().sum())

# Check for missing values in df_test
print("\nMissing values in df_test:")
print(df_test.isnull().sum())



Missing values in df_train:
source                       0
target                       0
edges                        0
circle_overlap               0
circle_jaccard_similarity    0
has_shared_core_circle       0
circle_adamic_adar_index     0
source_circle_count          0
target_circle_count          0
dtype: int64

Missing values in df_test:
source                       0
target                       0
edges                        0
circle_overlap               0
circle_jaccard_similarity    0
has_shared_core_circle       0
circle_adamic_adar_index     0
source_circle_count          0
target_circle_count          0
dtype: int64


### Model 1: GraphSAGE-Based GNN with Handcrafted Circle-Aware Features

In [ ]:
print(df_train.head())


   source  target  edges  circle_overlap  circle_jaccard_similarity  \
0    1945    2071      1               1                        0.5   
1     224     322      1               1                        1.0   
2    1833    1902      1               1                        1.0   
3    3226    3390      1               1                        1.0   
4    2835    2956      1               1                        1.0   

   has_shared_core_circle  circle_adamic_adar_index  source_circle_count  \
0                       1                  0.204484                  2.0   
1                       1                  0.204484                  1.0   
2                       1                  0.174517                  1.0   
3                       1                  0.184635                  1.0   
4                       1                  0.184635                  1.0   

   target_circle_count  
0                  1.0  
1                  1.0  
2                  1.0  
3               

In [ ]:
from sklearn.model_selection import train_test_split

# Split 10% of the training set as validation set
df_train_split, df_val = train_test_split(df_train, test_size=0.1, random_state=42)

# Convert to PyG data format
def create_pyg_data(df):
    edge_index = torch.tensor(df[["source", "target"]].values.T, dtype=torch.long)
    node_features = df.drop(columns=["source", "target", "edges"]).values
    x = torch.tensor(node_features, dtype=torch.float)
    y = torch.tensor(df["edges"].values, dtype=torch.long)
    return Data(x=x, edge_index=edge_index, y=y)

# Convert training, validation, and test sets
train_data = create_pyg_data(df_train_split)
val_data = create_pyg_data(df_val)
test_data = create_pyg_data(df_test)

print(train_data)
print(val_data)
print(test_data)

Data(x=[142938, 6], edge_index=[2, 142938], y=[142938])
Data(x=[15882, 6], edge_index=[2, 15882], y=[15882])
Data(x=[17648, 6], edge_index=[2, 17648], y=[17648])


In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize model & optimizer
model = GNN(in_channels=train_data.x.shape[1], hidden_channels=64, out_channels=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(train_data.x, train_data.edge_index).squeeze()
    loss = criterion(out, train_data.y.float())
    loss.backward()
    optimizer.step()
    return loss.item()

def validate():
    model.eval()
    with torch.no_grad():
        out = model(val_data.x, val_data.edge_index).squeeze()
        loss = criterion(out, val_data.y.float())
    return loss.item()

# Train for 100 epochs and compute validation loss
for epoch in range(100):
    loss = train()
    if epoch % 10 == 0:
        val_loss = validate()
        print(f"Epoch {epoch}, Train Loss: {loss:.4f}, Validation Loss: {val_loss:.4f}")

Epoch 0, Train Loss: 0.6813, Validation Loss: 0.6516
Epoch 10, Train Loss: 0.4895, Validation Loss: 0.4761
Epoch 20, Train Loss: 0.4373, Validation Loss: 0.4375
Epoch 30, Train Loss: 0.4319, Validation Loss: 0.4322
Epoch 40, Train Loss: 0.4272, Validation Loss: 0.4270
Epoch 50, Train Loss: 0.4262, Validation Loss: 0.4263
Epoch 60, Train Loss: 0.4257, Validation Loss: 0.4258
Epoch 70, Train Loss: 0.4255, Validation Loss: 0.4255
Epoch 80, Train Loss: 0.4254, Validation Loss: 0.4254
Epoch 90, Train Loss: 0.4252, Validation Loss: 0.4253


In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index).squeeze()
        preds = (torch.sigmoid(out) > 0.5).long().cpu().numpy()
        labels = data.y.cpu().numpy()

    roc_auc = roc_auc_score(labels, preds)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    return roc_auc, accuracy, precision, recall, f1

# **Validation Set Metrics**
val_metrics = evaluate(model, val_data)
print(f"\n**Validation Set Metrics:**")
print(f"ROC-AUC Score: {val_metrics[0]:.4f}")
print(f"Accuracy: {val_metrics[1]:.4f}")
print(f"Precision: {val_metrics[2]:.4f}")
print(f"Recall: {val_metrics[3]:.4f}")
print(f"F1 Score: {val_metrics[4]:.4f}")

# **Test Set Metrics**
test_metrics = evaluate(model, test_data)
print(f"\n**Test Set Metrics:**")
print(f"ROC-AUC Score: {test_metrics[0]:.4f}")
print(f"Accuracy: {test_metrics[1]:.4f}")
print(f"Precision: {test_metrics[2]:.4f}")
print(f"Recall: {test_metrics[3]:.4f}")
print(f"F1 Score: {test_metrics[4]:.4f}")



**Validation Set Metrics:**
ROC-AUC Score: 0.8082
Accuracy: 0.8090
Precision: 0.8722
Recall: 0.7200
F1 Score: 0.7889

**Test Set Metrics:**
ROC-AUC Score: 0.8101
Accuracy: 0.8101
Precision: 0.8795
Recall: 0.7186
F1 Score: 0.7909


## Model 2: Node2Vec-Initialized GraphSAGE for Link Prediction on Facebook Graph (Final Model)


In [ ]:
import pandas as pd

# Read the txt file
file_path = "/content/drive/Shareddrives/MLNS_Final_Facebook/data/facebook_combined.txt/facebook_combined.txt"

# Read data (assuming it's an edge list separated by spaces)
df = pd.read_csv(file_path, sep=" ", header=None, names=["source", "target"])

# Add an 'edges' column with value 1
df["edges"] = 1

# Display the first few rows of the data
print(df.head())  # Display in terminal
print(df.info())

   source  target  edges
0       0       1      1
1       0       2      1
2       0       3      1
3       0       4      1
4       0       5      1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88234 entries, 0 to 88233
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   source  88234 non-null  int64
 1   target  88234 non-null  int64
 2   edges   88234 non-null  int64
dtypes: int64(3)
memory usage: 2.0 MB
None


In [ ]:
import networkx as nx
import pandas as pd

# Read the edge file
edge_file = "/content/drive/Shared drives/MLNS_Final_Facebook/data/facebook_combined.txt/facebook_combined.txt"
df = pd.read_csv(edge_file, sep=" ", header=None, names=["source", "target"])

# Build an undirected graph
G = nx.Graph()
G.add_edges_from(df.values.tolist())

print(f"The graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

The graph has 4039 nodes and 88234 edges


In [ ]:
from node2vec import Node2Vec

# Train Node2Vec model
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Get embeddings for each node
import numpy as np
node_embeddings = []
node_id_map = {}  # Create a mapping from node to index
for i, node in enumerate(G.nodes()):
    node_id_map[node] = i
    node_embeddings.append(model.wv[str(node)])  # Nodes are treated as strings

node_features = np.array(node_embeddings)

Computing transition probabilities:   0%|          | 0/4039 [00:00<?, ?it/s]

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-geometric


Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 72.3 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.9 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

# Convert NetworkX graph to PyG format
G_int = nx.convert_node_labels_to_integers(G)  # Ensure nodes are consecutive integers
pyg_data = from_networkx(G_int)

# Set node features
pyg_data.x = torch.tensor(node_features, dtype=torch.float)

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [ ]:
import random

# All positive sample edges (with integer node IDs)
edges = list(G_int.edges())
random.shuffle(edges)

# Construct negative samples (randomly select non-existent edges)
all_nodes = list(G_int.nodes())
existing_edges = set([tuple(sorted(e)) for e in edges])
neg_edges = set()
while len(neg_edges) < len(edges):
    u, v = random.sample(all_nodes, 2)
    edge = tuple(sorted((u, v)))
    if edge not in existing_edges:
        neg_edges.add(edge)

# Convert to Tensor format
def edge_index_from_edges(edge_list):
    return torch.tensor(edge_list, dtype=torch.long).t().contiguous()

pos_edge_index = edge_index_from_edges(edges)
neg_edge_index = edge_index_from_edges(list(neg_edges))

# 80:20 split for test set
num_total = pos_edge_index.size(1)
split = int(num_total * 0.8)
train_pos, test_pos = pos_edge_index[:, :split], pos_edge_index[:, split:]
train_neg, test_neg = neg_edge_index[:, :split], neg_edge_index[:, split:]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.link_pred = nn.Linear(2 * hidden_channels, 1)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, edge_index):
        src, dst = edge_index
        edge_feat = torch.cat([z[src], z[dst]], dim=1)
        return torch.sigmoid(self.link_pred(edge_feat)).view(-1)


In [ ]:
import torch
from sklearn.metrics import roc_auc_score

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model
model = GNNModel(in_channels=node_features.shape[1], hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCELoss()

# Move data to GPU
pyg_data = pyg_data.to(device)
train_pos, train_neg = train_pos.to(device), train_neg.to(device)
test_pos, test_neg = test_pos.to(device), test_neg.to(device)

# Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    z = model.encode(pyg_data.x, pyg_data.edge_index)
    pos_pred = model.decode(z, train_pos)
    neg_pred = model.decode(z, train_neg)

    preds = torch.cat([pos_pred, neg_pred])
    labels = torch.cat([torch.ones_like(pos_pred), torch.zeros_like(neg_pred)])

    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 0.6869
Epoch 10, Loss: 0.3437
Epoch 20, Loss: 0.3193
Epoch 30, Loss: 0.3044
Epoch 40, Loss: 0.2963
Epoch 50, Loss: 0.2898
Epoch 60, Loss: 0.2833
Epoch 70, Loss: 0.2766
Epoch 80, Loss: 0.2721
Epoch 90, Loss: 0.2671


In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score

@torch.no_grad()
def evaluate_test(model, pos_edge_index, neg_edge_index, device):
    model.eval()
    z = model.encode(pyg_data.x, pyg_data.edge_index)

    pos_pred = model.decode(z, pos_edge_index).cpu()
    neg_pred = model.decode(z, neg_edge_index).cpu()

    preds = torch.cat([pos_pred, neg_pred])
    labels = torch.cat([torch.ones_like(pos_pred), torch.zeros_like(neg_pred)])

    pred_probs = preds.numpy()
    pred_labels = (pred_probs > 0.5).astype(int)
    true_labels = labels.numpy()

    auc = roc_auc_score(true_labels, pred_probs)
    acc = accuracy_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)

    return auc, acc, recall, f1


In [ ]:
auc, acc, recall, f1 = evaluate_test(model, test_pos, test_neg, device)

print(f"Test ROC-AUC: {auc:.4f}")
print(f"Test Accuracy: {acc:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


Test ROC-AUC: 0.9475
Test Accuracy: 0.8973
Test Recall: 0.9392
Test F1 Score: 0.9014
